In [1]:
# try older xarray version for debugging
# !mamba install xarray=2022.06.0 -y

In [10]:
import json
import gcsfs
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
import zarr

from utils import smooth_inputs
from cm26_utils import load_and_combine_cm26

# set up gcs filesystem
# 👇 replace with your key 
with open( '/home/jovyan/KEYS/pangeo-forge-ocean-transport-4967-347e2048c5a1.json') as token_file:
    token = json.load(token_file)
fs = gcsfs.GCSFileSystem(token=token)

### 🚧 temporary fix. Write the already written store out again to avoid long loading times.

In [4]:
# xr.open_dataset(output_mapper, engine='zarr')

In [24]:
ds_ref = load_and_combine_cm26(fs)

ref_time = ds_ref[['time']].load()
ref_time

Load Data
Align in time
Interpolating ocean velocities
Regrid Atmospheric Data
Modify units
Mask nans
Calculate relative wind


<xarray.Dataset>
Dimensions:  (time: 7305)
Coordinates:
  * time     (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
Data variables:
    *empty*
Attributes:
    regrid_method:  bilinear

In [11]:
%%time
output_path = 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr'
mapper = fs.get_mapper(output_path)

CPU times: user 31 µs, sys: 11 µs, total: 42 µs
Wall time: 46 µs


In [23]:
def fix_time_chunks(mapper):
    ds = xr.open_dataset(mapper, engine='zarr', consolidated=True, chunks={})
    replace_time = ds[['time']].load()
    print(replace_time)
    
    # delete time
    g = zarr.open_group(mapper)
    del g['time']
    
    zarr.consolidate_metadata(mapper)
    
    #TODO: show metadata before and after!

In [31]:
# fs.rm('ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr/.zmetadata')

In [46]:
# delete time
g = zarr.open_group(mapper)
del g['time']
list(g.arrays())

[('algo', <zarr.core.Array '/algo' (3,) <U8>),
 ('evap', <zarr.core.Array '/evap' (3, 4, 7305, 3600, 2700) float32>),
 ('qh', <zarr.core.Array '/qh' (3, 4, 7305, 3600, 2700) float32>),
 ('ql', <zarr.core.Array '/ql' (3, 4, 7305, 3600, 2700) float32>),
 ('smoothing', <zarr.core.Array '/smoothing' (4,) <U13>),
 ('taux', <zarr.core.Array '/taux' (3, 4, 7305, 3600, 2700) float32>),
 ('tauy', <zarr.core.Array '/tauy' (3, 4, 7305, 3600, 2700) float32>),
 ('xt_ocean', <zarr.core.Array '/xt_ocean' (3600,) float64>),
 ('yt_ocean', <zarr.core.Array '/yt_ocean' (2700,) float64>)]

In [49]:
zarr.consolidate_metadata(mapper)

<zarr.hierarchy.Group '/'>

In [51]:
with fs.open(mapper.root+'/.zmetadata') as f:
    meta = json.load(f)
meta['metadata']

{'.zattrs': {'regrid_method': 'bilinear'},
 '.zgroup': {'zarr_format': 2},
 'algo/.zarray': {'chunks': [3],
  'compressor': {'blocksize': 0,
   'clevel': 5,
   'cname': 'lz4',
   'id': 'blosc',
   'shuffle': 1},
  'dtype': '<U8',
  'fill_value': None,
  'filters': None,
  'order': 'C',
  'shape': [3],
  'zarr_format': 2},
 'algo/.zattrs': {'_ARRAY_DIMENSIONS': ['algo']},
 'evap/.zarray': {'chunks': [1, 1, 1, 3600, 2700],
  'compressor': {'blocksize': 0,
   'clevel': 5,
   'cname': 'lz4',
   'id': 'blosc',
   'shuffle': 1},
  'dtype': '<f4',
  'fill_value': 'NaN',
  'filters': None,
  'order': 'C',
  'shape': [3, 4, 7305, 3600, 2700],
  'zarr_format': 2},
 'evap/.zattrs': {'_ARRAY_DIMENSIONS': ['algo',
   'smoothing',
   'time',
   'xt_ocean',
   'yt_ocean'],
  'filter_scale': 50},
 'qh/.zarray': {'chunks': [1, 1, 1, 3600, 2700],
  'compressor': {'blocksize': 0,
   'clevel': 5,
   'cname': 'lz4',
   'id': 'blosc',
   'shuffle': 1},
  'dtype': '<f4',
  'fill_value': 'NaN',
  'filters': N

In [ ]:
ref_time.to_zarr(mapper, mode='w')

In [ ]:
zarr.consolidate_metadata(mapper)

In [ ]:
with fs.open(mapper.root+'/.zmetadata') as f:
    meta = json.load(f)
meta['metadata']['time/.zarray']

In [33]:
%%time
xr.open_dataset(mapper, engine='zarr', consolidated=True, chunks={})

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f04a11c8430>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/_weakrefset.py", line 39, in _remove
KeyboardInterrupt: 

KeyboardInterrupt



## 

In [4]:
import xarray as xr
xr.__version__

'2022.6.0'

In [5]:
dask.__version__

'2022.10.0'

In [ ]:
xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]
python-bits: 64
OS: Linux
OS-release: 5.10.133+
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: C.UTF-8
LANG: C.UTF-8
LOCALE: ('en_US', 'UTF-8')
libhdf5: 1.12.2
libnetcdf: 4.8.1

xarray: 2022.6.0
pandas: 1.5.0
numpy: 1.23.3
scipy: 1.9.1
netCDF4: 1.6.1
pydap: installed
h5netcdf: 1.0.2
h5py: 3.7.0
Nio: None
zarr: 2.13.3
cftime: 1.6.2
nc_time_axis: 1.4.1
PseudoNetCDF: None
rasterio: 1.3.2
cfgrib: 0.9.10.2
iris: None
bottleneck: 1.3.5
dask: 2022.10.0
distributed: 2022.10.0
matplotlib: 3.6.1
cartopy: 0.21.0
seaborn: 0.12.0
numbagg: None
fsspec: 2022.8.2
cupy: None
pint: 0.19.2
sparse: 0.13.0
flox: 0.6.0
numpy_groupies: 0.9.19
setuptools: 65.5.0
pip: 22.3
conda: None
pytest: 7.1.3
IPython: 8.5.0
sphinx: None


/srv/conda/envs/notebook/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [20]:
# hacky stuff (but it works!)
n_workers = 10
n_threads = 2 # I think 12 is the max?
# with time chunks as 3 this needs to be set down. 4 seemed to work but had some memory to spare.


from dask_gateway import Gateway
gateway = Gateway()

# close existing clusters
open_clusters = gateway.list_clusters()
print(list(open_clusters))
if len(open_clusters)>0:
    for c in open_clusters:
        cluster = gateway.connect(c.name)
        cluster.shutdown()  

options = gateway.cluster_options()

# set the options programatically, or through their HTML repr
options.worker_memory = 16
options.worker_cores = n_threads 

options.environment = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)

print('start')
# Create a cluster with those options
cluster = gateway.new_cluster(options)
client = cluster.get_client()
cluster.scale(n_workers)


# with dask.config.set({"distributed.scheduler.worker-saturation": 1.0}):
#     cluster = LocalCluster()
#     client = Client(cluster)

client

[]
start


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.90e442c3ef7141228cc1da2b03bb1e60/status,


In [8]:
fs.ls('ocean-transport-group/scale-aware-air-sea/flux-output')

['ocean-transport-group/scale-aware-air-sea/flux-output/cm26.zarr',
 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_sample.zarr',
 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split.zarr',
 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr']

In [9]:
fs.ls('ocean-transport-group/scale-aware-air-sea/preprocessed')

['ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed.zarr',
 'ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_new.zarr',
 'ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_split.zarr',
 'ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_test_split.zarr']

In [10]:
# TODO: Remove old ones once new works

In [11]:
# import zarr
# zarr.consolidate_metadata(mapper)

In [12]:
# import zarr
# mapper_smoothed = fs.get_mapper('ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_split.zarr')
# zarr.open_group(mapper_smoothed)

## Issue for later: Seems like the long opening times are due to xarray?
Debug further where it takes to long

In [13]:
%%time
mapper_smoothed = fs.get_mapper('ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_split.zarr')
ds_smoothed = xr.open_dataset(mapper_smoothed, engine='zarr', chunks='auto', consolidated=True)
ds_smoothed


KeyboardInterrupt



In [ ]:
%%time
mapper_flux = fs.get_mapper('ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr')
ds_flux = xr.open_dataset(mapper_flux, engine='zarr', chunks='auto', consolidated=True)
# extract the filter scale used (TODO: Check that it is the same for all variables?
filter_scale = ds_flux.ql.attrs['filter_scale']
ds_flux

CPU times: user 16.8 s, sys: 1.39 s, total: 18.2 s
Wall time: 6min 47s


,Array,Chunk
Bytes,3.10 TiB,37.08 MiB
Shape,"(3, 4, 7305, 3600, 2700)","(1, 1, 1, 3600, 2700)"
Count,2 Graph Layers,87660 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,3.10 TiB,37.08 MiB
Shape,"(3, 4, 7305, 3600, 2700)","(1, 1, 1, 3600, 2700)"
Count,2 Graph Layers,87660 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [ ]:
ds_original = load_and_combine_cm26(fs)
ds_original

Load Data
Align in time
Interpolating ocean velocities
Regrid Atmospheric Data
Modify units
Mask nans
Calculate relative wind


<xarray.Dataset>
Dimensions:       (time: 7305, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time          (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [ ]:
# combine all fields
ds = xr.merge([ds_smoothed, ds_original])
ds

<xarray.Dataset>
Dimensions:                (time: 7305, xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * time                   (time) object 0181-01-01 12:00:00 ... 0200-12-31 1...
  * xt_ocean               (xt_ocean) float64 -279.9 -279.8 ... 79.85 79.95
  * yt_ocean               (yt_ocean) float64 -81.11 -81.07 ... 89.94 89.98
Data variables: (12/17)
    q_ref_smoothed         (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    slp_smoothed           (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    surface_temp_smoothed  (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    t_ref_smoothed         (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    u_relative_smoothed    (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    v_relative_smoothed    (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    ...                     ...
    wind                   (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp           (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean                (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean                (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [ ]:
ds_small_scale = (ds_flux.sel(smoothing='smooth_full') - ds_flux).isel(smoothing=slice(1,None))
ds_small_scale

,Array,Chunk
Bytes,2.32 TiB,37.08 MiB
Shape,"(3, 7305, 3600, 2700, 3)","(1, 1, 3600, 2700, 1)"
Count,8 Graph Layers,65745 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,2.32 TiB,37.08 MiB
Shape,"(3, 7305, 3600, 2700, 3)","(1, 1, 3600, 2700, 1)"
Count,8 Graph Layers,65745 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


## 🚨 Temporary patch with the smoothing...
Need to adjust this once I have written out the data without smoothing after the fluxes (this should only be applied on the mean)

In [ ]:
def smooth_inputs_dataset(ds:xr.Dataset, dims:list, filter_scale:int, timedim:str='time') -> xr.Dataset:
    """Wrapper that filters a whole dataset, generating a wet_mask from the nanmask of the first timestep (if time is present)."""
    ds_out = xr.Dataset()
    for var in ds.data_vars:
        da = ds[var]
        if timedim in da.dims:
            mask_da = da.isel({timedim:0})
        else:
            mask_da = da
            
        wet_mask = ~np.isnan(mask_da)
        ds_out[var] = smooth_inputs(da, wet_mask, dims, filter_scale)
    return ds_out

In [19]:
test = ds_small_scale.isel(time=slice(0,12))
test.load()


KeyboardInterrupt



In [22]:
ds_small_scale_mean = ds_small_scale.mean('time').load()
ds_small_scale_mean

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.9/ssl.py", line 1310, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLEOFError: EOF occurred in violation of protocol (_ssl.c:1129)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle_read()
  File "/srv/conda/envs/notebook/lib/python3.9/site

CancelledError: ('mean_agg-aggregate-8271dc6340fd02662cc6a0be97a91193', 0, 0, 0, 2)

In [ ]:
def small_scale_plot(ds_without_time, filter_scale):
    # this should eventually be applied to smooth_all too
    smooth_vars = ['smooth_vel', 'smooth_tracer']

    datasets = []
    for sm in ds_small_scale_mean.smoothing:
        ds_partial = ds_small_scale_mean.sel(smoothing=sm)
        if sm in smooth_vars:
            datasets.append(smooth_inputs_dataset(ds_partial, ['xt_ocean', 'yt_ocean'], filter_scale))
        else:
            datasets.append(ds_partial)

    ds_plot = xr.concat(datasets, dim='smoothing')
    return ds_plot
    

In [ ]:
small_scale_plot(ds_small_scale_mean)

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.9/ssl.py", line 1310, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1129)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle